# Parse preprint full-texts

Contents:
1. Introduction
2. 

Check if downloaded .tex files contain fulltext. Some are just supplementary files for the submission.  If they contain \documentclass. 
Each submission folder should contain one \documentclass file. 

In [2]:
import os, sys, re, requests, subprocess, glob, multiprocessing, time

Each submission folder may contain 1 or more .tex files. We need to identify which .tex file is the full-text preprint.

In [3]:
def get_preprints():
    base_path = 'latex'
    submissions = 0
    tex_count = 0
    empties = []
    preprints = []
    missed = []

    # Walk through tars
    for idx, tar_folder in enumerate(os.listdir(base_path)):
        tar_path = base_path + '/' + tar_folder
        if not os.path.isdir(tar_path):
            continue
        submission_dirs = os.listdir(tar_path)
        submissions += len(submission_dirs)
        # Walk through each submission
        for submission in submission_dirs:
            submission_path = tar_path + '/' + submission
            if not os.path.isdir(submission_path):
                submissions -= 1
                continue
            arxiv_id = os.path.basename(submission_path)
            texs = glob.glob(submission_path + '/**/*.tex', recursive=True)
            tex_count += len(texs)
            
            # If submission is empty, note & skip
            if len(texs) == 0:
                empties.append(arxiv_id)
                continue
            # Otherwise get the preprint
            else:
                preprint_path = getPreprint(submission_path, texs)
                if preprint_path:
                    preprints.append(preprint_path)
                else:
                    missed.append(arxiv_id)
                    
    print('Submissions: ' + str(submissions))
    print('Preprints: ' + str(len(preprints)))
    print('Non-empty submissions missing preprints: ' + str(len(missed)))
    print('Empty submissions: ' + str(len(empties)))
    print('Texs: ' + str(tex_count))
    print(str(missed))
    
    return preprints

def getPreprint(submission_path, texs):
    preprint = None
    
    # If submission contains only one file, this is the preprint
    if len(texs) == 1:
        preprint = texs[0]
    # If submission contains ms.tex or main.tex, this is the preprint
    elif 'ms.tex' in texs:
        preprint = submission_path + '/' + 'ms.tex'
    elif 'main.tex' in texs:
        preprint = submission_path + '/' + 'main.tex'
    # Otherwise, iterate through each .tex looking for \documentclass or \documentstyle
    else: 
        for tex_path in texs: 
            with open(tex_path, 'rb') as f: 
                data = f.readlines()
                r = re.compile(b'(.*\\\\documentclass.*)|(.*\\\\documentstyle.*)')
                if len(list(filter(r.match, data))) > 0:
                    preprint = tex_path
                    break
    
    return preprint

In [4]:
preprints = get_preprints()

Submissions: 89908
Preprints: 89630
Non-empty submissions missing preprints: 7
Empty submissions: 271
Texs: 125484
['1105.1087', '1211.4277', '1304.7762', '1308.6483', '1409.3422', '1606.06791', '1607.01189']


In [5]:
len(preprints)

89630

arxiv-vanity is unable to render these 7 papers as well. I'll just leave them for now.

Convert each preprint via multiprocessing.

Before you can begin multiprocessing, you need to pick which sections of code to multiprocess. These sections of code must meet the following criteria:

1. Must not be reliant on previous outcomes. True.

2. Does not need to be executed in a particular order. True.

3. Does not return anything that would need to be accessed later in the code. True.

In [ ]:
def convert_to_xml(tex_path):
    '''
    Converts TEX file to XML.
    '''
    
    print('--> Conversion beginning for {} to XML'.format(tex_path))
    # Convert given .tex to XML
    filename, file_extension = os.path.splitext(os.path.basename(tex_path))
    xml_path = 'xml/' + filename + '.xml'
    
    if os.path.exists(xml_path):
        print(os.path.basename(tex_path) + ' has already been converted.')
    elif file_extension == '.tex':
        subprocess.call(['latexml', '--dest=' + xml_path, tex_path])
        print('--> Conversion complete for {} to XML'.format(tex_path))

def start_conversion():
    starttime = time.time()
    pool = multiprocessing.Pool(processes=4)
    # Specify a timeout so that this pool can be interrupted
    pool.map_async(convert_to_xml, preprints).get(9999999)
    pool.close()
    print('That took {} seconds'.format(time.time() - starttime))

start_conversion()

--> Conversion beginning for latex/arXiv_src_1008_005/1008.3737/stats_ms4_pdf.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3779/preprint.tex to XML
--> Conversion beginning for latex/arXiv_src_1107_001/1107.0722/manuscrip3.tex to XML
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7015/DA+DAH-archive.tex to XML
--> Conversion complete for latex/arXiv_src_1111_009/1111.7015/DA+DAH-archive.tex to XML
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7019/BD-arXiv.tex to XML
--> Conversion complete for latex/arXiv_src_1008_005/1008.3737/stats_ms4_pdf.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3740/nstarB-f2.tex to XML
--> Conversion complete for latex/arXiv_src_1101_005/1101.3779/preprint.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3780/ms.tex to XML
ms.tex has already been converted.
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3781/bhnodisk.tex to XML
--> Conversion complete f

--> Conversion beginning for latex/arXiv_src_1107_001/1107.0732/1107.0732.tex to XML
--> Conversion complete for latex/arXiv_src_1101_005/1101.3870/0294380.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3875/nsi_hints_v2_arxiv.tex to XML
--> Conversion complete for latex/arXiv_src_1008_005/1008.3823/vennes.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3828/astroph.tex to XML
--> Conversion complete for latex/arXiv_src_1111_009/1111.7071/BrakingApJ.tex to XML
--> Conversion beginning for latex/arXiv_src_1111_009/1111.7072/IRFlare_sjschmidt.tex to XML
--> Conversion complete for latex/arXiv_src_1101_005/1101.3875/nsi_hints_v2_arxiv.tex to XML
--> Conversion beginning for latex/arXiv_src_1101_005/1101.3876/AstroPH_IRlums-finalrevised.tex to XML
--> Conversion complete for latex/arXiv_src_1008_005/1008.3828/astroph.tex to XML
--> Conversion beginning for latex/arXiv_src_1008_005/1008.3848/13544.tex to XML
--> Conversion complete for latex/arXi

NOTE:
If I use `pool.map(convert_to_xml, preprints)`, I cannot kill this. So this is why I used `pool.map_async()` with a timeout specified. 
https://stackoverflow.com/questions/1408356/keyboard-interrupts-with-pythons-multiprocessing-pool

If I ever use pool.map accidentally - this is how to kill the stupid Medusa-spawning workers: https://stackoverflow.com/questions/25415104/kill-python-multiprocessing-pool


In [228]:
PROCESSES = 4

def process_tasks(task_queue):
    print('Processing tasks...')
    while not task_queue.empty():
        preprint = task_queue.get()
        convert_to_xml(preprint)
    return True

def add_tasks(task_queue, number_of_tasks):
    print('Adding tasks...')
    for num in range(len(preprints)):
        task_queue.put(preprints[num])
    return task_queue

def run():
    empty_task_queue = multiprocessing.Queue()
    full_task_queue = add_tasks(empty_task_queue, NUMBER_OF_TASKS)
    processes = []
    print(f'Running with {PROCESSES} processes!')
    start = time.time()
    for n in range(PROCESSES):
        p = multiprocessing.Process(
            target=process_tasks, args=(full_task_queue,))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
    print(f'Time taken = {time.time() - start:.3f}')

run()

Adding tasks...


KeyboardInterrupt: 

Convert all to XML:

Attempt using more cores:

It is faster to use all 8 cores. So I will do the XML conversion using all 8 cores. 

First I need to confirm the main file in each repository. 
- If it doesn't contain a .bbl file, I need to add it to the bbl_lack folder. Later. Set aside and skip.
- If it doesn't contain a file, I need to retrieve it again. Later. Set aside and skip. 

I will look at each submission folder, check xml to see if a file exists with its name. If not, I will go into the submission folder to check each file if it contains \\documentclass. If it does, grab it and convert it. Break out of loop. 

Now call the functions, with conversion process.

I have 8 cores: https://superuser.com/questions/1101311/how-many-cores-does-my-mac-have/1101314#1101314
4 physical and 4 logical. 

In [165]:
def guess_extension_from_headers(h):
    """
    Given headers from an ArXiV e-print response, try and guess what the file
    extension should be.
    Based on: https://arxiv.org/help/mimetypes
    """
    if h.get('content-type') == 'application/pdf':
        return '.pdf'
    if h.get('content-encoding') == 'x-gzip' and h.get('content-type') == 'application/postscript':
        return '.ps.gz'
    if h.get('content-encoding') == 'x-gzip' and h.get('content-type') == 'application/x-eprint-tar':
        return '.tar.gz'
    # content-encoding is x-gzip but this appears to normally be a lie - it's
    # just plain text
    if h.get('content-type') == 'application/x-eprint':
        return '.tex'
    if h.get('content-encoding') == 'x-gzip' and h.get('content-type') == 'application/x-dvi':
        return '.dvi.gz'
    return None

def arxiv_id_to_source_url(arxiv_id):
    # This URL is normally a tarball, but sometimes something else.
    # ArXiV provides a /src/ URL which always serves up a tarball,
    # but if we used this, we'd have to untar the file to figure out
    # whether it's renderable or not. By using the /e-print/ endpoint
    # we can figure out straight away whether we should bother rendering
    # it or not.
    # https://arxiv.org/help/mimetypes has more info
    return 'https://arxiv.org/e-print/' + arxiv_id

def download_source_file(arxiv_id):
    """
    Download the LaTeX source of this paper and returns as ContentFile.
    """
    source_url = arxiv_id_to_source_url(arxiv_id)
    res = requests.get(source_url)
    res.raise_for_status()
    extension = guess_extension_from_headers(res.headers)
    if not extension:
        raise DownloadError("Could not determine file extension from "
                            "headers: Content-Type: {}; "
                            "Content-Encoding: {}".format(
                                res.headers.get('content-type'),
                                res.headers.get('content-encoding')))
    with open(arxiv_id + extension, 'wb+') as f:
        f.write(res.content)
        print('Created ' + arxiv_id + extension)

download_source_file('1010.3382')

Created 1010.3382.tar.gz
